In [ ]:
# 0. Import the libraries
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from pmdarima import auto_arima

In [ ]:
# 1. Load the dataset
file_path = r"C:\Users\c.hakker\OneDrive - VISTA college\Senior Stuff\Opleiding Data science\Data\merged_tables.csv"
df = pd.read_csv(file_path)

In [ ]:
# 2. Filter for the specific branch 'G Handel'
branch_name = 'G Handel'
df = df[df['BedrijfstakkenBranchesSBI2008'].str.strip().str.lower() == branch_name.strip().lower()]

In [ ]:
# 3. Verify and process the 'Year' and 'Quarter' columns for date alignment
if 'Year' not in df.columns or 'Quarter' not in df.columns:
    raise KeyError("The 'Year' and 'Quarter' columns are required in the dataset.")

In [ ]:
# 4. Create a 'Date' column representing the start of each quarter
df['Year'] = df['Year'].astype(int)
df['Quarter'] = df['Quarter'].astype(int)
df['Date'] = pd.PeriodIndex.from_fields(year=df['Year'], quarter=df['Quarter'], freq='Q').to_timestamp()

In [ ]:
# 5. Drop rows where 'Date' could not be created
df = df.dropna(subset=['Date'])

# Set 'Date' as the index for time-series analysis
df.set_index('Date', inplace=True, drop=True)

# Remove duplicate index values if they exist
df = df.loc[~df.index.duplicated(keep='first')]

In [ ]:
# 6. Set the frequency explicitly
try:
    df = df.asfreq('QS')  # Infer the quarterly frequency and set it explicitly
except ValueError as e:
    print(f"Warning: {e}. Frequency could not be set explicitly.")

In [ ]:
# 7. Define parameters for SARIMAX model and validate features
branch_name = 'g handel'
target_column = '80072ned_Ziekteverzuimpercentage_1'
exog_columns = [
    '83451NED_ArbeidsvolumeMetSeizoenscorrectie_3',
    '85663NED_CaoLonenPerMaandExclBijzBeloningen_1'
]

# Add lagged and differenced features
df['CaoLonen_Lag1'] = df['85663NED_CaoLonenPerMaandExclBijzBeloningen_1'].shift(1)
df['Arbeidsvolume_Diff'] = df['83451NED_ArbeidsvolumeMetSeizoenscorrectie_3'].diff()

# Calculate correlation matrix for numeric columns
numeric_df = df.select_dtypes(include=[np.number])
corr_matrix = numeric_df.corr()

# Select additional features based on correlation
additional_exog = [
    '83451NED_BanenMetSeizoenscorrectie_1',  # Example high-correlation feature
    'CaoLonen_Lag1',  # Lagged feature
    'Arbeidsvolume_Diff'  # Differenced feature
]

selected_exog_columns = exog_columns + additional_exog
selected_exog_columns = [col for col in selected_exog_columns if col in df.columns]  # Ensure columns exist

# Debug: Print selected columns
print("Final Selected Exogenous Columns:", selected_exog_columns)

Final Selected Exogenous Columns: ['83451NED_ArbeidsvolumeMetSeizoenscorrectie_3', '85663NED_CaoLonenPerMaandExclBijzBeloningen_1', '83451NED_BanenMetSeizoenscorrectie_1', 'CaoLonen_Lag1', 'Arbeidsvolume_Diff']


In [ ]:
# 8. Prepare the target and exogenous variables for training and testing
y = df[target_column]
exog = df[selected_exog_columns]

In [ ]:
# 9. Split data into training (2008-2021) and testing (2022)
train_end_year = 2021
test_year = 2022
train_df = df[df['Year'] <= train_end_year]
test_df = df[df['Year'] == test_year]

y_train = train_df[target_column]
exog_train = train_df[selected_exog_columns]
y_test = test_df[target_column]
exog_test = test_df[selected_exog_columns]

In [ ]:
# 10. Handle missing values
exog_train.fillna(0, inplace=True)  # Handle NaN in exog_train
exog_test.fillna(0, inplace=True)  # Handle NaN in exog_test

C:\Users\c.hakker\AppData\Local\Temp\ipykernel_14540\373449355.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exog_train.fillna(0, inplace=True)  # Handle NaN in exog_train
C:\Users\c.hakker\AppData\Local\Temp\ipykernel_14540\373449355.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exog_test.fillna(0, inplace=True)  # Handle NaN in exog_test


In [ ]:
# 11. Normalize the exogenous variables
scaler_exog = StandardScaler()
exog_train_scaled_array = scaler_exog.fit_transform(exog_train)
exog_test_scaled_array = scaler_exog.transform(exog_test)

# Re-create scaled DataFrame with proper column names
exog_train_scaled = pd.DataFrame(
    exog_train_scaled_array,
    index=exog_train.index,
    columns=selected_exog_columns
)
exog_test_scaled = pd.DataFrame(
    exog_test_scaled_array,
    index=exog_test.index,
    columns=selected_exog_columns
)

# Log transform the target variable
y_train_log = np.log(y_train + 1)
y_test_log = np.log(y_test + 1)

In [ ]:
# 12. Fit the SARIMAX model
print(f"\nFitting SARIMAX model for branch '{branch_name}'...")
model = sm.tsa.SARIMAX(
    y_train_log,
    exog=exog_train_scaled,
    order=(1, 1, 1),  # Adjust as needed
    seasonal_order=(1, 1, 0, 4),  # Adjust as needed
    enforce_stationarity=False,
    enforce_invertibility=False
)
results = model.fit(disp=False)

# Print model summary
print(results.summary())


Fitting SARIMAX model for branch 'g handel'...
                                       SARIMAX Results                                        
Dep. Variable:     80072ned_Ziekteverzuimpercentage_1   No. Observations:                   56
Model:                 SARIMAX(1, 1, 1)x(1, 1, [], 4)   Log Likelihood                  99.151
Date:                                Thu, 28 Nov 2024   AIC                           -180.302
Time:                                        21:25:40   BIC                           -163.844
Sample:                                    01-01-2008   HQIC                          -174.136
                                         - 10-01-2021                                         
Covariance Type:                                  opg                                         
                                                    coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------

c:\Users\c.hakker\OneDrive - VISTA college\Senior Stuff\Opleiding Data science\uwv\.venv\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [ ]:
# 13. Make predictions for the test year and generate forecast for 2023
# Predict for the test set
forecast_test = results.get_forecast(steps=len(y_test_log), exog=exog_test_scaled)
y_pred_test_log = forecast_test.predicted_mean
y_pred_test = np.exp(y_pred_test_log) - 1  # Back-transform predictions to original scale

# Forecast for 2023
forecast_steps = 4  # 4 quarters
# Generate 2023 exogenous variables (reusing the last available exog values; replace with real data if available)
exog_forecast_scaled = exog_test_scaled.iloc[-1:].values  # Duplicate last row of exog
exog_forecast_scaled = np.tile(exog_forecast_scaled, (forecast_steps, 1))  # Repeat for 4 quarters

# Create forecast for 2023
forecast_2023 = results.get_forecast(steps=forecast_steps, exog=exog_forecast_scaled)
forecast_mean_log = forecast_2023.predicted_mean
forecast_mean = np.exp(forecast_mean_log) - 1  # Back-transform forecast to original scale

# Create index for forecast (2023-Q1 to 2023-Q4)
forecast_index = pd.date_range(start="2023-01-01", periods=forecast_steps, freq='QS')

# Calculate MAE for the test set
q1_mask = test_df['Quarter'] == 1
mae_q1 = mean_absolute_error(y_test[q1_mask], y_pred_test[q1_mask])
mae_all = mean_absolute_error(y_test, y_pred_test)

print(f"\nMAE for Q1 {test_year} for branch '{branch_name}': {mae_q1:.4f}")
print(f"MAE for all quarters of {test_year} for branch '{branch_name}': {mae_all:.4f}")


MAE for Q1 2022 for branch 'g handel': 0.3441
MAE for all quarters of 2022 for branch 'g handel': 0.3012


In [ ]:
# 14. Visualization with Forecast for 2023
fig = go.Figure()

# Add actual sick leave percentage line (test data)
fig.add_trace(go.Scatter(
    x=y_test.index,
    y=y_test,
    mode='lines+markers',
    name='Actual',
    line=dict(color='#0078d2', width=2)
))

# Add predictions line for test period
fig.add_trace(go.Scatter(
    x=y_test.index,
    y=y_pred_test,
    mode='lines+markers',
    name='Predictions',
    line=dict(color='orange', width=2, dash='dash')
))

# Add forecast line for 2023
fig.add_trace(go.Scatter(
    x=forecast_index,
    y=forecast_mean,
    mode='lines+markers',
    name='Forecast',
    line=dict(color='green', width=2, dash='dot')
))

# Add MAE values as text annotations
fig.add_annotation(
    xref="paper", yref="paper", x=0.00, y=1.15, showarrow=False,
    text=f"MAE for all four quarters: {mae_all:.4f}",
    font=dict(size=12, color="black")
)

fig.add_annotation(
    xref="paper", yref="paper", x=0.00, y=1.20, showarrow=False,
    text=f"MAE for next quarter (Q1): {mae_q1:.4f}",
    font=dict(size=12, color="black")
)

# Set layout and design for better visualization
fig.update_layout(
    title=f'Sick Leave Predictions and Forecast - Trade Branch',
    xaxis_title='Date',
    yaxis_title='Sick Leave Percentage',
    plot_bgcolor='white',
    xaxis=dict(
        showgrid=False,
        tickformat="%Y-%m",  # Year-Month format for clearer date labeling
        range=[y_test.index.min(), forecast_index[-1]]  # Extend range to include 2023
    ),
    yaxis=dict(
        showgrid=True, gridcolor='lightgrey', showline=True, linewidth=0.5, linecolor='black'
    ),
    legend=dict(
        x=0.91, y=1.5, traceorder="normal"
    ),
    font=dict(family="Roboto", size=14),
    margin=dict(l=50, r=50, t=80, b=50),
    width=1100, height=500
)

# Show Plot
fig.show()

# Optionally Save Plot as HTML
fig.write_html("C:/Users/c.hakker/Downloads/Sick_leave_predict_forecast_G_Handel.html")
